# Minimal example using omero-py
#### Laura Cooper 26/01/2021

## Import packages

In [ ]:
from omero.gateway import BlitzGateway
import getpass
import numpy as np
import matplotlib.pyplot as plt

## Helper functions

Many functions and code snipets can be found on the OMERO site \
https://docs.openmicroscopy.org/omero/latest/developers/Python.html

In [ ]:
def print_obj(obj, indent=0):
    """
    Helper method to display info about OMERO objects.
    Not all objects will have a "name" or owner field.
    """
    print("""%s%s:%s  Name:"%s" (owner=%s)""" % (
        " " * indent,
        obj.OMERO_CLASS,
        obj.getId(),
        obj.getName(),
        obj.getOwnerOmeName()))

## Connect to OMERO - Must be connected to the VPN

In [ ]:
username = input('Username: ') #Request user to enter OMERO username
password = getpass.getpass(prompt='Password: ') #Request user to enter OMERO password
conn = BlitzGateway(username, password, host='camdu.warwick.ac.uk', port=4064) #Use details to connect to OMERO
conn.connect() #Returns True when connected

## Check your ID and Default Group

In [ ]:
my_id = conn.getUser().getId()
default_group_name = conn.getEventContext().groupName
default_group_id = conn.getEventContext().groupId
print("My user ID is", my_id)
print("My default group is %s with ID %s" % (default_group_name, default_group_id))

## List Datasets and Images

In [ ]:
print("\nList Datasets: \n", "=" * 50)
datasets = conn.getObjects("Dataset", opts={'owner': my_id}) #list all of my datasets 
for obj in datasets:
    print_obj(obj)

You can do the same thing for other object types: project, dataset, image, screen, plate, plateacquisition, acquisition, well, roi, shape, experimenter, experimentergroup, originalfile, fileset, annotation.

NB. getObjects() returns a generator not a list

In [ ]:
print("\nList Images: \n", "=" * 50)
images = conn.getObjects("Image", opts={'owner': my_id, 'dataset': 13801}) #list all image in dataset
for obj in images:
    print_obj(obj)

## View or Save an image

In [ ]:
image = conn.getObject("Image", '312305')
print(image.getName(), image.getDescription())
# Retrieve information about an image.
print("Image dimensions [x,y,z,c,t]: [%s %s %s %s %s]" % (
    image.getSizeX(),
    image.getSizeY(),
    image.getSizeZ(),
    image.getSizeC(),
    image.getSizeT()))

In [ ]:
z = image.getSizeZ() / 2 #Middle z plane
t = 0 #First time point
rendered_image=image.renderImage(z,t) #Rendering settings from OMERO.web
rendered_image.show() #Opens pop up
rendered_image.save('my_image.png') #Save image to current folder

## Import Image as Numpy Array

Helper function

In [ ]:
def load_plane(image,z,c,t):
    """
    Load a 2D-plane as a numpy array
    param image: The image
    param z: z plane
    param c: channel
    param t: time point
    """
    pixels = image.getPrimaryPixels()
    return pixels.getPlane(z, c, t)

In [ ]:
c=1
plane=load_plane(image,z,c,t)
plt.imshow(plane)

In [ ]:
tile = (0, 0, image.getSizeX(), image.getSizeY())     # x, y, width, height of tile
# list of [ (0,0,0,(x,y,w,h)), (1,0,0,(x,y,w,h)), (2,0,0,(x,y,w,h))... ]
zct_list = [(iz, c, t, tile) for iz in range(10)]
pixels = image.getPrimaryPixels()
print("\nZ stack of tiles:")
planes = pixels.getTiles(zct_list)
for i, p in enumerate(planes):
    print("Tile:", zct_list[i], " min:", p.min(),\
        " max:", p.max(), " sum:", p.sum())

In [ ]:
def get_z_stack(img, c=0, t=0):
    """
    Convert OMERO image object to numpy array
    Input: img  OMERO image object
           c    number of colour channls
           t    number of time steps
    """
    zct_list = [(z, c, t) for z in range(img.getSizeZ())] #Set dimensions of image
    pixels = img.getPrimaryPixels()
    return np.array(list(pixels.getPlanes(zct_list))) #Read in data one plane at a time

In [ ]:
stack = get_z_stack(image,c,t)
stack

Using getTiles or getPlanes returns a generator of data (not all the data in hand)

Use pixels.getPlanes() to import a hyperstack